In [1]:
from preprocessing_temp import prep_window_generator
import tensorflow as tf


import neptune
from neptune.integrations.tensorflow_keras import NeptuneCallback

import os


In [2]:
window_generator = prep_window_generator()


In [8]:
from copy import deepcopy
from typing import List


class Params:
    lstm_layer_count: int
    lstm_sizes: List[int]
    lstm_bidirectional: bool
    lstm_return_sequences: bool
    pooling_type: str
    kernel_sizes: List[int]
    dense_layer_count: int
    conv_layer_sizes: List[int]
    dropout: float
    input_width: int
    label_width: int
    batch_size: int
    loss_function: str

    def __init__(
        self,
        lstm_layer_count=1,
        lstm_sizes=[32],
        lstm_bidirectional=False,
        lstm_return_sequences=False,
        pooling_type="MaxPooling",
        kernel_sizes=[5, 5, 5],
        dense_layer_count=1,
        dense_layer_sizes=[512],
        conv_layer_sizes=[128, 256, 256],
        dropout=0.2,
        input_width=7 * 24,
        label_width=1 * 24,
        batch_size=32,
        loss_function="mse",
    ):
        self.lstm_layer_count = lstm_layer_count
        self.lstm_sizes = lstm_sizes
        self.lstm_bidirectional = lstm_bidirectional
        self.lstm_return_sequences = lstm_return_sequences
        self.pooling_type = pooling_type
        self.kernel_sizes = kernel_sizes
        self.dense_layer_count = dense_layer_count
        self.dense_layer_sizes = dense_layer_sizes
        self.conv_layer_sizes = conv_layer_sizes
        self.dropout = dropout
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.loss_function = loss_function

    def to_name(self, epochs: int):
        name = "cnn_"
        if self.lstm_bidirectional:
            name += "bi"
        name += "lstm_"
        name += str(int(self.input_width / 24)) + "days_history_"
        name += str(int(self.label_width / 24)) + "days_pred_"
        name += str(epochs) + "_epochs"
        name += "_" + str(self.lstm_layer_count) + "lstm-layer"
        name += "_" + ",".join([str(i) for i in self.lstm_sizes])
        if self.lstm_return_sequences:
            name += "_ret-seq"
        if self.pooling_type != "MaxPooling":
            name += "_" + "pooling-" + self.pooling_type
        if self.dropout != 0.2:
            name += "_" + "dropout-0-" + str(int(self.dropout * 10))

        return name

    def to_dict(self):
        self_dict = deepcopy(self.__dict__)
        for key, value in self_dict.items():
            if isinstance(value, list):
                self_dict[key] = str(value)
        return self_dict


In [4]:
def init_model(params: Params):
    num_features = 1
    init_kernel_size = params.kernel_sizes[0]
    cnn_lstm_model = tf.keras.models.Sequential()
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    cnn_lstm_model.add(tf.keras.layers.Normalization())
    cnn_lstm_model.add(tf.keras.layers.Lambda(lambda x: x[:, -init_kernel_size:, :]))
    # Shape => [batch, 1, conv_units]
    cnn_lstm_model.add(
        tf.keras.layers.Conv1D(
            params.conv_layer_sizes[0], activation="relu", kernel_size=params.kernel_sizes[0], padding="same"
        )
    )
    if params.pooling_type == "MaxPooling":
        cnn_lstm_model.add(tf.keras.layers.MaxPooling1D())
    else:
        cnn_lstm_model.add(tf.keras.layers.AveragePooling1D())
    cnn_lstm_model.add(tf.keras.layers.BatchNormalization())  # TODO: useful to put it here?
    cnn_lstm_model.add(
        tf.keras.layers.Conv1D(
            params.conv_layer_sizes[1], activation="relu", kernel_size=params.kernel_sizes[1], padding="same"
        )
    )
    if params.pooling_type == "MaxPooling":
        cnn_lstm_model.add(tf.keras.layers.MaxPooling1D())
    else:
        cnn_lstm_model.add(tf.keras.layers.AveragePooling1D())
    cnn_lstm_model.add(tf.keras.layers.BatchNormalization())  # TODO: useful to put it here?
    cnn_lstm_model.add(
        tf.keras.layers.Conv1D(
            params.conv_layer_sizes[2], activation="relu", kernel_size=params.kernel_sizes[2], padding="same"
        )
    )

    cnn_lstm_model.add(tf.keras.layers.Reshape((-1, 256)))

    if params.lstm_layer_count != len(params.lstm_sizes):
        raise ValueError("lstm_layer_count must be same as the length of lstm_sizes!")

    # LSTM
    for i in range(params.lstm_layer_count):
        if i == params.lstm_layer_count:
            lstm_layer2 = tf.keras.layers.LSTM(params.lstm_sizes[i], return_sequences=params.lstm_return_sequences)
            if params.lstm_bidirectional:
                cnn_lstm_model.add(tf.keras.layers.Bidirectional(lstm_layer2))
            else:
                cnn_lstm_model.add(lstm_layer2)
        else:
            lstm_layer1 = tf.keras.layers.LSTM(params.lstm_sizes[i], return_sequences=True)
            if params.lstm_bidirectional:
                cnn_lstm_model.add(tf.keras.layers.Bidirectional(lstm_layer1))
            else:
                cnn_lstm_model.add(lstm_layer1)

    for i in range(params.dense_layer_count):
        cnn_lstm_model.add(tf.keras.layers.Dense(params.dense_layer_sizes[i]))
        cnn_lstm_model.add(tf.keras.layers.Dropout(params.dropout))

    cnn_lstm_model.add(
        tf.keras.layers.Dense(params.label_width * num_features, kernel_initializer=tf.initializers.zeros())
    )
    # Shape => [batch, out_steps, features] -> for each prediction step one neuron
    cnn_lstm_model.add(tf.keras.layers.Reshape([params.label_width, num_features]))
    return cnn_lstm_model


In [9]:
from typing import Dict


run_params: Dict[str, Params] = {}
# run_params["base_run"] = Params()
run_params["lstm_size_64"] = Params(lstm_sizes=[64])
run_params["two_lstm_size_32"] = Params(lstm_layer_count=2, lstm_sizes=[32, 32])
run_params["two_lstm_size_64"] = Params(lstm_layer_count=2, lstm_sizes=[64, 64])
run_params["bidirectional_lstm"] = Params(lstm_bidirectional=True)
run_params["bidirectional_two_lstm"] = Params(lstm_bidirectional=True, lstm_layer_count=2, lstm_sizes=[32, 32])
run_params["average_pooling"] = Params(pooling_type="AveragePooling")
run_params["kernel_sizes_adjusted_to_hours"] = Params(kernel_sizes=[24, 12, 6])
run_params["two_dense_layer"] = Params(dense_layer_count=2, dense_layer_sizes=[512, 256])
run_params["dropout_0.4"] = Params(dropout=0.3)
run_params["dropout_0.4"] = Params(dropout=0.4)
run_params["dropout_0.5"] = Params(dropout=0.5)
run_params["dropout_0.6"] = Params(dropout=0.6)
run_params["loss_huber"] = Params(loss_function="huber")


In [6]:
import logging

logger = logging.getLogger("run_logger")
fileHandler = logging.FileHandler("run.logs.txt")
logger.addHandler(fileHandler)


In [10]:
for run_id, params in run_params.items():
    logger.error("Start run: " + run_id)
    model = init_model(params)

    loss = tf.keras.losses.MeanSquaredError()
    if params.loss_function == "huber":
        loss = tf.keras.losses.huber
    model.compile(loss=loss, optimizer="Adam", metrics=[tf.keras.metrics.MeanAbsoluteError()])

    name = params.to_name(100)
    temp_name = name
    append_index = 0
    while os.path.exists("../models/" + temp_name):
        append_index += 1
        temp_name = name + "_" + str(append_index)
    if append_index != 0:
        name += "_" + str(append_index)

    run = neptune.init_run(
        project="data-mining-team2/initial-model-tests",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4Mzg2ZWZmYi05YzRlLTQ3ODYtOWE1NC1mNDM4OTM1ZjNlOTkifQ==",
        custom_run_id=run_id,
        name=name,
        source_files=["./initial_model_tests.ipynb"],
    )

    run["parameters"] = params.to_dict()

    neptune_cbk = NeptuneCallback(run=run, base_namespace="training")

    try:
        for epoch_count_factor in range(1, 5):
            epoch_count = 50 * epoch_count_factor
            history = model.fit(
                window_generator.train,
                epochs=50,
                validation_data=window_generator.val,
                callbacks=[neptune_cbk],
            )

            eval_metrics = model.evaluate(window_generator.test, verbose=0)
            for j, metric in enumerate(eval_metrics):
                run[f"eval/epoch_{epoch_count}/{model.metrics_names[j]}"] = metric

            name = params.to_name(epoch_count)
            temp_name = name
            append_index = 0
            while os.path.exists("../models/" + temp_name):
                append_index += 1
                temp_name = name + "_" + str(append_index)
            if append_index != 0:
                name += "_" + str(append_index)

            model.save("../models/" + name, include_optimizer=False)

            run["model_names/" + str(epoch_count) + "epochs"] = name
    except Exception as e:
        logger.error(e)
        pass
    run.stop()
    logger.error("Finished run: " + run_id)


https://app.neptune.ai/data-mining-team2/initial-model-tests/e/IMT-12
Epoch 1/50
728/728 [==============================] - 9s 9ms/step - loss: 383.8957 - mean_absolute_error: 12.9362 - val_loss: 339.6826 - val_mean_absolute_error: 11.4839
Epoch 2/50
728/728 [==============================] - 6s 8ms/step - loss: 323.3899 - mean_absolute_error: 11.8764 - val_loss: 309.3740 - val_mean_absolute_error: 11.0878
Epoch 3/50
728/728 [==============================] - 6s 8ms/step - loss: 309.8830 - mean_absolute_error: 11.5545 - val_loss: 291.1024 - val_mean_absolute_error: 10.7857
Epoch 4/50
728/728 [==============================] - 6s 8ms/step - loss: 302.7831 - mean_absolute_error: 11.3988 - val_loss: 305.1350 - val_mean_absolute_error: 11.1603
Epoch 5/50
728/728 [==============================] - 6s 8ms/step - loss: 297.1704 - mean_absolute_error: 11.2983 - val_loss: 292.8333 - val_mean_absolute_error: 10.7309
Epoch 6/50
728/728 [==============================] - 6s 8ms/step - loss: 287.91

INFO:tensorflow:Assets written to: ../models/cnn_lstm_7days_history_1days_pred_50_epochs_1lstm-layer_64\assets


INFO:tensorflow:Assets written to: ../models/cnn_lstm_7days_history_1days_pred_50_epochs_1lstm-layer_64\assets


Epoch 1/50
728/728 [==============================] - 7s 9ms/step - loss: 100.1003 - mean_absolute_error: 7.0280 - val_loss: 191.8872 - val_mean_absolute_error: 8.9636
Epoch 2/50
728/728 [==============================] - 7s 9ms/step - loss: 101.3216 - mean_absolute_error: 7.0594 - val_loss: 204.1670 - val_mean_absolute_error: 9.0583
Epoch 3/50
728/728 [==============================] - 6s 9ms/step - loss: 100.1558 - mean_absolute_error: 7.0184 - val_loss: 184.9387 - val_mean_absolute_error: 8.9125
Epoch 4/50
728/728 [==============================] - 6s 9ms/step - loss: 98.7602 - mean_absolute_error: 6.9828 - val_loss: 188.8231 - val_mean_absolute_error: 8.8937
Epoch 5/50
728/728 [==============================] - 6s 9ms/step - loss: 96.7457 - mean_absolute_error: 6.9204 - val_loss: 187.1244 - val_mean_absolute_error: 8.8034
Epoch 6/50
728/728 [==============================] - 6s 9ms/step - loss: 95.9240 - mean_absolute_error: 6.8862 - val_loss: 189.6389 - val_mean_absolute_error: 8.

INFO:tensorflow:Assets written to: ../models/cnn_lstm_7days_history_1days_pred_100_epochs_1lstm-layer_64\assets


...

Epoch 1/50
728/728 [==============================] - 13s 17ms/step - loss: 4.7773 - mean_absolute_error: 5.2483 - val_loss: 7.7326 - val_mean_absolute_error: 8.2129
Epoch 2/50
728/728 [==============================] - 12s 17ms/step - loss: 4.7793 - mean_absolute_error: 5.2504 - val_loss: 7.7988 - val_mean_absolute_error: 8.2786
Epoch 3/50
728/728 [==============================] - 12s 17ms/step - loss: 4.7744 - mean_absolute_error: 5.2451 - val_loss: 7.6347 - val_mean_absolute_error: 8.1140
Epoch 4/50
728/728 [==============================] - 12s 17ms/step - loss: 4.7829 - mean_absolute_error: 5.2539 - val_loss: 7.6216 - val_mean_absolute_error: 8.1008
Epoch 5/50
728/728 [==============================] - 12s 17ms/step - loss: 4.7631 - mean_absolute_error: 5.2339 - val_loss: 7.7559 - val_mean_absolute_error: 8.2359
Epoch 6/50
728/728 [==============================] - 12s 17ms/step - loss: 4.7361 - mean_absolute_error: 5.2068 - val_loss: 7.6383 - val_mean_absolute_error: 8.1183
Epoc

INFO:tensorflow:Assets written to: ../models/cnn_lstm_7days_history_1days_pred_200_epochs_1lstm-layer_32_2\assets


INFO:tensorflow:Assets written to: ../models/cnn_lstm_7days_history_1days_pred_200_epochs_1lstm-layer_32_2\assets


Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/data-mining-team2/initial-model-tests/e/IMT-23/metadata


ERROR:run_logger:Finished run: loss_huber
